<h1>Imports</h1>

In [1]:
from pathlib import Path

import numpy as np
import sklearn as slearn
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn import svm
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import fbeta_score
from sklearn.metrics import classification_report

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

from imblearn.combine import SMOTETomek

import glob
import os

Using TensorFlow backend.


In [2]:
pd.set_option("display.max_columns", 100)
%matplotlib inline

In [3]:
RANDOM_SEED = 6

<h1>Dataset Importing</h1>

In [4]:
DATA_PATH = Path.cwd() / "../../datasets"

In [5]:
training_df = pd.read_csv(DATA_PATH / "train.csv", index_col="tripid")
training_df.head()

additional_fare  duration  meter_waiting  meter_waiting_fare  \
tripid                                                                    
189123628             10.5     834.0           56.0              0.0000   
189125358             10.5     791.0           47.0              0.0000   
189125719             10.5    1087.0           80.0              0.0000   
189127273             10.5     598.0          271.0             15.6638   
189128020              NaN       NaN            NaN                 NaN   

           meter_waiting_till_pickup     pickup_time       drop_time  \
tripid                                                                 
189123628                       64.0  11/1/2019 0:20  11/1/2019 0:34   
189125358                      134.0  11/1/2019 0:56  11/1/2019 1:09   
189125719                       61.0  11/1/2019 1:08  11/1/2019 1:26   
189127273                       68.0  11/1/2019 2:27  11/1/2019 2:37   
189128020                        NaN  11/1/2019 3:34  11/1/2019 3:51   

           pick_lat  pick_lon  drop_lat  drop_lon    fare    label  
tripid                                                              
189123628   6.86252   79.8993   6.90330   79.8783  270.32  correct  
189125358   6.88589   79.8984   6.91373   79.8923  197.85  correct  
189125719   6.90839   79.8651   6.93669   79.9146  301.64  correct  
189127273   6.92570   79.8895   6.92748   79.8971   82.30  correct  
189128020   6.87441   79.8615   6.84478   79.9290  358.39  correct

In [6]:
training_df = training_df.dropna(subset=['fare'])

<h1>Data Preprocessing</h1>

<h2>Data Conversions</h2>

In [7]:
# Converting date time objects from Objects to datetime64
training_df['pickup_time'] = pd.to_datetime(training_df['pickup_time'], format="%m/%d/%Y %H:%M")
training_df['drop_time'] = pd.to_datetime(training_df['drop_time'], format="%m/%d/%Y %H:%M")

<h2>Adding Features</h2>
Adding features deemed useful from Feature Engineering

In [8]:
def dist_from_coordinates(lat1, lon1, lat2, lon2):
  R = 6371  # Earth radius in km

  #conversion to radians
  d_lat = np.radians(lat2-lat1)
  d_lon = np.radians(lon2-lon1)

  r_lat1 = np.radians(lat1)
  r_lat2 = np.radians(lat2)

  #haversine formula
  a = np.sin(d_lat/2.) **2 + np.cos(r_lat1) * np.cos(r_lat2) * np.sin(d_lon/2.)**2

  haversine = 2 * R * np.arcsin(np.sqrt(a))

  return haversine

In [9]:
training_df = training_df.assign(timeOfDay=pd.cut(training_df.pickup_time.dt.hour,[-1, 8, 20, 24],labels=['dawn','day', 'night']))

In [10]:
new_column = []                    #empty column for distance
for index,row in training_df.iterrows():
  lat1 = row['pick_lat'] #first row of location.lat column here
  lon1 = row['pick_lon'] #first row of location.long column here
  lat2 = row['drop_lat'] #second row of location.lat column here
  lon2 = row['drop_lon'] #second row of location.long column here
  value = dist_from_coordinates(lat1, lon1, lat2, lon2)  #get the distance
  new_column.append(value)   #append the empty list with distance values

training_df.insert(4,"distance",new_column)

In [11]:
# Dataset after adding Columns
training_df

additional_fare  duration  meter_waiting  meter_waiting_fare  \
tripid                                                                    
189123628             10.5     834.0           56.0            0.000000   
189125358             10.5     791.0           47.0            0.000000   
189125719             10.5    1087.0           80.0            0.000000   
189127273             10.5     598.0          271.0           15.663800   
189128020              NaN       NaN            NaN                 NaN   
189129552             10.5    3407.0          182.0            0.000000   
189132829             10.5    1246.0          487.0            0.000000   
189135103             10.5    1333.0          295.0           17.198500   
189139296             10.5     360.0           80.0            4.664000   
189138671             10.5    1539.0          588.0           33.986400   
189135814             10.5      26.0           19.0            0.000000   
189140969             10.5      31.0           20.0            0.000000   
189144528             10.5    2011.0          199.0            0.000000   
189147041             10.5    1260.0           30.0            0.000000   
189145904             10.5     335.0           10.0            0.583000   
189146174             10.5    4221.0         1608.0           84.214600   
189150541             10.5    1472.0          219.0           12.699976   
189153229             10.5     284.0            0.0            0.000000   
189151585             10.5    2435.0          381.0            0.000000   
189157464             10.5     891.0          170.0            0.000000   
189157476             10.5    3176.0          390.0            0.000000   
189157559             10.5     693.0          280.0            9.883800   
189159951              5.0     850.0          143.0            8.336900   
189158726             10.5    1443.0           59.0            0.000000   
189160756             10.5     358.0           66.0            0.000000   
189161377             10.5     644.0          638.0           14.796800   
189163162             10.5     618.0          128.0            7.398400   
189164454             10.5    1407.0          210.0            0.000000   
189164880             10.5     575.0           23.0            0.000000   
189165701              5.0    1588.0          302.0           17.606600   
...                    ...       ...            ...                 ...   
213743810             10.5    2122.0          248.0           14.458400   
213747503             10.5    2417.0          183.0           10.577400   
213751620              5.0     179.0            2.0            0.116600   
213749432             10.5    1184.0          219.0           12.658200   
213754172             10.5    1049.0            5.0            0.000000   
213759061             10.5     413.0           11.0            0.000000   
213761623             10.5    2905.0          389.0           22.484200   
213776255              5.0     358.0           30.0            1.749000   
213774316             10.5     451.0           83.0            4.838900   
213774593             10.5    1155.0          100.0            0.000000   
213775846             10.5     965.0          374.0           21.804200   
213774642             10.5     649.0           34.0            1.971977   
213774742             10.5     485.0           74.0            0.000000   
213779882             10.5     995.0          248.0           14.365427   
213782178             10.5     685.0           32.0            0.000000   
213782382             10.5    1326.0           20.0            1.156000   
213784559             10.5    1125.0           85.0            0.000000   
213784596             10.5     831.0           27.0            0.000000   
213787716             10.5     733.0           43.0            0.000000   
213789771             30.5    1009.0          178.0            0.000000   
213795752             40.5    1859.0

<h1>Model Training</h1>
<h2> Part 1 : Fare Predictor </h2>
Predict fare using features

In [12]:
# Extract Correctly predicted fares
correct_training_df = training_df[training_df['label'] == 'correct']

In [13]:
# Extract the useful features deemed in feature engineering
features_predictor = correct_training_df[['additional_fare', 'duration', 'meter_waiting', 'meter_waiting_fare',
       'meter_waiting_till_pickup','distance','timeOfDay']]
fare_prediction = correct_training_df[['fare']]

In [14]:
features_predictor.dtypes

additional_fare               float64
duration                      float64
meter_waiting                 float64
meter_waiting_fare            float64
meter_waiting_till_pickup     float64
distance                      float64
timeOfDay                    category
dtype: object

In [15]:
numerical_features = features_predictor.columns[features_predictor.dtypes == "float64"].values
categorical_features = features_predictor.columns[features_predictor.dtypes == "category" ].values

In [16]:
X_train, X_eval, y_train, y_eval = train_test_split(features_predictor, fare_prediction, test_size=0.3, shuffle=True)

In [17]:

## chain numerical preprocessing into a pipeline object
numeric_preprocessing_steps = Pipeline([
    ('simple_imputer', SimpleImputer(strategy='median'))
])

non_numeric_preprocessing_steps = Pipeline([
    ('simple_imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot_encoder', OneHotEncoder(handle_unknown='ignore')),
#     ('label_encoder', LabelEncoder())
])

## create preprocessor stage of the final pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('non_numeric', non_numeric_preprocessing_steps, categorical_features),
        ('numeric', numeric_preprocessing_steps, numerical_features)
    ],
    remainder = 'drop'
)

In [18]:
# Set of new columns
new_columns = ['isDawn','isDay','isNight','additional_fare', 'duration', 'meter_waiting', 'meter_waiting_fare',
       'meter_waiting_till_pickup', 'distance']

In [19]:
# Transform training data columns 
preprocessed_train_features = preprocessor.fit_transform(X_train)
preprocessed_train_features_data_frame = pd.DataFrame(data=preprocessed_train_features, columns=new_columns)

In [20]:
# Transform tvalidation data columns 
preprocessed_test_features = preprocessor.fit_transform(X_eval)
preprocessed_test_features_data_frame = pd.DataFrame(data=preprocessed_test_features, columns=new_columns)

In [21]:
# Choosing the Predictor
estimator = RandomForestRegressor()

# Train the model
estimator.fit(preprocessed_train_features, y_train)

# Predict for the evaluation set
y_pred = estimator.predict(preprocessed_test_features_data_frame)
slearn.metrics.mean_squared_error(y_pred, y_eval)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


19954.267306565354

In [22]:
# training using split data done prepare features for training with full dataset !! Optional !!

required_features = training_df[['additional_fare', 'duration', 'meter_waiting', 'meter_waiting_fare',
       'meter_waiting_till_pickup','distance','timeOfDay']]
preprocessed_full_features = preprocessor.fit_transform(required_features)
preprocessed_full_features_data_frame = pd.DataFrame(data=preprocessed_full_features, columns=new_columns)

In [23]:
predicted_prices = estimator.predict(preprocessed_full_features_data_frame)

In [24]:
training_df.insert(13,'predicted_price',predicted_prices)

In [25]:
# Training dataset after new features inserted
training_df

additional_fare  duration  meter_waiting  meter_waiting_fare  \
tripid                                                                    
189123628             10.5     834.0           56.0            0.000000   
189125358             10.5     791.0           47.0            0.000000   
189125719             10.5    1087.0           80.0            0.000000   
189127273             10.5     598.0          271.0           15.663800   
189128020              NaN       NaN            NaN                 NaN   
189129552             10.5    3407.0          182.0            0.000000   
189132829             10.5    1246.0          487.0            0.000000   
189135103             10.5    1333.0          295.0           17.198500   
189139296             10.5     360.0           80.0            4.664000   
189138671             10.5    1539.0          588.0           33.986400   
189135814             10.5      26.0           19.0            0.000000   
189140969             10.5      31.0           20.0            0.000000   
189144528             10.5    2011.0          199.0            0.000000   
189147041             10.5    1260.0           30.0            0.000000   
189145904             10.5     335.0           10.0            0.583000   
189146174             10.5    4221.0         1608.0           84.214600   
189150541             10.5    1472.0          219.0           12.699976   
189153229             10.5     284.0            0.0            0.000000   
189151585             10.5    2435.0          381.0            0.000000   
189157464             10.5     891.0          170.0            0.000000   
189157476             10.5    3176.0          390.0            0.000000   
189157559             10.5     693.0          280.0            9.883800   
189159951              5.0     850.0          143.0            8.336900   
189158726             10.5    1443.0           59.0            0.000000   
189160756             10.5     358.0           66.0            0.000000   
189161377             10.5     644.0          638.0           14.796800   
189163162             10.5     618.0          128.0            7.398400   
189164454             10.5    1407.0          210.0            0.000000   
189164880             10.5     575.0           23.0            0.000000   
189165701              5.0    1588.0          302.0           17.606600   
...                    ...       ...            ...                 ...   
213743810             10.5    2122.0          248.0           14.458400   
213747503             10.5    2417.0          183.0           10.577400   
213751620              5.0     179.0            2.0            0.116600   
213749432             10.5    1184.0          219.0           12.658200   
213754172             10.5    1049.0            5.0            0.000000   
213759061             10.5     413.0           11.0            0.000000   
213761623             10.5    2905.0          389.0           22.484200   
213776255              5.0     358.0           30.0            1.749000   
213774316             10.5     451.0           83.0            4.838900   
213774593             10.5    1155.0          100.0            0.000000   
213775846             10.5     965.0          374.0           21.804200   
213774642             10.5     649.0           34.0            1.971977   
213774742             10.5     485.0           74.0            0.000000   
213779882             10.5     995.0          248.0           14.365427   
213782178             10.5     685.0           32.0            0.000000   
213782382             10.5    1326.0           20.0            1.156000   
213784559             10.5    1125.0           85.0            0.000000   
213784596             10.5     831.0           27.0            0.000000   
213787716             10.5     733.0           43.0            0.000000   
213789771             30.5    1009.0          178.0            0.000000   
213795752             40.5    1859.0

<h2>Part 2: Training the classifier</h2>

In [26]:
classifier_features = training_df[['predicted_price','fare']]
classifier_label = training_df[['label']]
mapping = {'correct': 1, 'incorrect': 0}
classifier_label = classifier_label.replace({'label':mapping})

In [27]:
classifier_label

label
tripid          
189123628      1
189125358      1
189125719      1
189127273      1
189128020      1
189129552      1
189132829      1
189135103      1
189139296      1
189138671      1
189135814      0
189140969      0
189144528      1
189147041      0
189145904      1
189146174      1
189150541      1
189153229      1
189151585      1
189157464      1
189157476      1
189157559      1
189159951      1
189158726      1
189160756      0
189161377      0
189163162      1
189164454      1
189164880      1
189165701      1
...          ...
213743810      1
213747503      1
213751620      1
213749432      1
213754172      1
213759061      1
213761623      1
213776255      1
213774316      1
213774593      1
213775846      1
213774642      1
213774742      1
213779882      1
213782178      1
213782382      1
213784559      1
213784596      1
213787716      1
213789771      1
213795752      0
213799350      1
213800748      1
213802197      1
213802222      1
213803193      1
213812756      1
213813930      1
213815405      1
213817296      1

[17039 rows x 1 columns]

In [28]:
X_train, X_eval, y_train, y_eval = train_test_split(classifier_features, classifier_label, test_size=0.3, shuffle=True)

In [29]:
classifier = RandomForestClassifier(random_state=42)
## Train the model
classifier.fit(X_train, y_train)

# Predict for the evaluation set
print("Training Accuracy: %.2f" % (classifier.score(X_eval, y_eval)*100), "%")
y_pred = classifier.predict(X_eval)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


Training Accuracy: 95.27 %


In [30]:
confusion_matrix = confusion_matrix(y_eval, y_pred)
confusion_matrix

array([[ 306,  158],
       [  84, 4564]])

In [31]:
print(classification_report(y_eval, y_pred))

              precision    recall  f1-score   support

           0       0.78      0.66      0.72       464
           1       0.97      0.98      0.97      4648

    accuracy                           0.95      5112
   macro avg       0.88      0.82      0.85      5112
weighted avg       0.95      0.95      0.95      5112



<h1>Model Validation</h1>

<h2>Loading the test data set</h2>

In [32]:
test_set = pd.read_csv(DATA_PATH / "test.csv", index_col="tripid")
test_set.head()

additional_fare  duration  meter_waiting  meter_waiting_fare  \
tripid                                                                    
213284604             10.5       924             42              2.4486   
213286352             10.5      4249             20              0.0000   
213293973             10.5      1552            255              2.6588   
213294622             10.5       462             16              0.0000   
213298687             10.5       814            392             12.3692   

           meter_waiting_till_pickup    pickup_time      drop_time  pick_lat  \
tripid                                                                         
213284604                        148  2/1/2020 0:38  2/1/2020 0:53   6.83454   
213286352                         91  2/1/2020 1:02  2/1/2020 2:13   6.91168   
213293973                         23  2/1/2020 5:02  2/1/2020 5:28   6.92145   
213294622                        198  2/1/2020 5:30  2/1/2020 5:38   6.77433   
213298687                         69  2/1/2020 7:00  2/1/2020 7:14   6.97968   

           pick_lon  drop_lat  drop_lon     fare  
tripid                                            
213284604   79.8750   6.77490   79.8840   289.27  
213286352   79.8723   6.55091   79.9706  1912.70  
213293973   79.8478   6.90539   79.8989   394.00  
213294622   79.9416   6.80401   79.9407   154.32  
213298687   79.9130   6.98875   79.8914   147.47

In [33]:
test_set.isna().sum()

additional_fare              0
duration                     0
meter_waiting                0
meter_waiting_fare           0
meter_waiting_till_pickup    0
pickup_time                  0
drop_time                    0
pick_lat                     0
pick_lon                     0
drop_lat                     0
drop_lon                     0
fare                         0
dtype: int64

<h2>Feature Addition for the Test Dataset</h2>

In [34]:
test_set['pickup_time'] = pd.to_datetime(test_set['pickup_time'], format="%m/%d/%Y %H:%M")
test_set['drop_time'] = pd.to_datetime(test_set['drop_time'], format="%m/%d/%Y %H:%M")

In [35]:
test_set = test_set.assign(timeOfDay=pd.cut(test_set.pickup_time.dt.hour,[-1, 8, 20, 24],labels=['dawn','day', 'night']))

In [36]:
new_column = []                    #empty column for distance
for index,row in test_set.iterrows():
  lat1 = row['pick_lat'] #first row of location.lat column here
  lon1 = row['pick_lon'] #first row of location.long column here
  lat2 = row['drop_lat'] #second row of location.lat column here
  lon2 = row['drop_lon'] #second row of location.long column here
  value = dist_from_coordinates(lat1, lon1, lat2, lon2)  #get the distance
  new_column.append(value)   #append the empty list with distance values

test_set .insert(4,"distance",new_column)

In [37]:
test_set

additional_fare  duration  meter_waiting  meter_waiting_fare  \
tripid                                                                    
213284604             10.5       924             42            2.448600   
213286352             10.5      4249             20            0.000000   
213293973             10.5      1552            255            2.658800   
213294622             10.5       462             16            0.000000   
213298687             10.5       814            392           12.369200   
213299545             10.5      2495            351           16.530800   
213302332             10.5      1108            454           23.929200   
213302671             10.5      2737            320           18.496000   
213305594             10.5      1154             29            0.000000   
213305134             10.5      1372            277           16.046498   
213306304             10.5      2808            760           44.308000   
213308616             10.5       780             54            3.148200   
213309839             10.5      5632           1186           68.580489   
213310068             10.5       368              3            0.000000   
213309032             10.5      1028            154            8.901200   
213310105             10.5      1677            101            0.000000   
213311364             10.5      1505             84            0.000000   
213309929             10.5      1742            119            6.924990   
213310221             10.5      1612             70            0.000000   
213312453             10.5       500             36            0.000000   
213313921             30.5      1334            213            0.000000   
213314038             10.5       712            134            0.000000   
213314867             10.5       363             10            0.000000   
213316124              5.0       261             32            1.865600   
213315491             10.5       932            321            0.000000   
213315522             10.5       872             48            2.774400   
213317715              5.0       277              0            0.000000   
213316458             10.5      1554              4            0.000000   
213316409             10.5      2832            629            0.000000   
213317742             10.5       273             28            1.618400   
...                    ...       ...            ...                 ...   
222825708             10.5       815             10            0.000000   
222826396             10.5      1043             90            0.000000   
222826895             10.5      1955            658            0.000000   
222826635             10.5      1478            332            0.000000   
222827460             10.5      2658             23            0.000000   
222828746             10.5       665             10            0.000000   
222830021             10.5      2270           1517           88.441100   
222830132             10.5       468             33            0.000000   
222830743             10.5       118             70            0.000000   
222831987             10.5      8842            510           29.733000   
222833285             10.5      1243            119            0.000000   
222833707             10.5      1206            201            0.000000   
222832962             10.5       740             30            1.734000   
222835856             10.5      2043             74            0.000000   
222837330             10.5       794             66            3.814800   
222837378             10.5       195              0            0.000000   
222837360             10.5      3405            152            8.785600   
222840348             10.5      1579             30            0.000000   
222849083             10.5       597            192           11.193600   
222850789             10.5      2871           1441           83.289800   
222851642             10.5       390

In [38]:
test_features = test_set[['additional_fare', 'duration', 'meter_waiting', 'meter_waiting_fare',
       'meter_waiting_till_pickup','distance','timeOfDay']]
preprocessed_test_features = preprocessor.fit_transform(test_features)
preprocessed_test_features_data_frame = pd.DataFrame(data=preprocessed_test_features, columns=new_columns)

<h2>Fare prediction and correctness prediction using Test Dataset</h2>

In [39]:
test_probs = estimator.predict(preprocessed_test_features_data_frame)

In [40]:
test_set.insert(10,'predicted_price',test_probs)

In [41]:
classifier_test_features = test_set[['predicted_price','fare']]

In [42]:
predicted_labels = classifier.predict(classifier_test_features)

<h2>Writing to the Submission File</h2>

In [43]:
submission_set = pd.read_csv(DATA_PATH / "sample_submission.csv", index_col="tripid")
submission_set.head()

submission_set['prediction']= predicted_labels

In [44]:
%%javascript
var kernel = IPython.notebook.kernel;
var thename = window.document.getElementById("notebook_name").innerHTML;
var command = "theNotebook = " + "'"+thename+"'";
kernel.execute(command);

<IPython.core.display.Javascript object>

In [45]:
filename = '../../submissions/'+theNotebook+'/teamCluster_submission_{%i}.csv'
dirname = '../../submissions/'+theNotebook
fileversion = 1

if not os.path.exists(dirname):
    os.makedirs(dirname)
while glob.glob(filename.replace('{%i}',str(fileversion))) :
    fileversion+=1
submission_set.to_csv(filename.replace('{%i}',str(fileversion)), index=True)
print("Completed!")

Completed!


In [46]:
submission_set['prediction'].idxmin()

213293973

In [47]:
submission_set['prediction'].value_counts()

1    8015
0     561
Name: prediction, dtype: int64